In [7]:
### Library Import Initialization

import numpy as np
import math
from math import *
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [4]:
### Function to Import Stock Tickers and Calculate Final Stock Price
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
S_0

[*********************100%%**********************]  1 of 1 completed


136.2899932861328

In [5]:
### Sigma Calculation 
def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

sigma = compute_sigma(stock_data)
sigma

GOOG    0.326178
dtype: float64

In [8]:
# Cumulative standard normal distribution
def cdf(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

### Compute phi Function
def phi(S_0, T, y, H, I, r, b, sigma):
    '''
    ϕ(S,T,y,H,I) represents the price of an American option with a continuous barrier. It incorporates the effects of early exercise 
    and the presence of a barrier, which can be either up-and-out, up-and-in, down-and-out, or down-and-in, depending on the 
    relative positions of the barrier level and the current asset price
    The function takes the following parameters:
    S: Current price of the underlying asset
    T: Time to expiration of the option
    y: The "nu" parameter, or gamma, which is often related to the cost of carry or dividend yield
    H: The barrier level
    I: The rebate, which is a cash payment given if the barrier is hit
    '''
    d = (np.log(S_0/H) + (b + ((y - 1/2) * sigma**2)) * T) / (sigma * np.sqrt(T))
    d_2 = (np.log(I**2/(S_0*H)) + (b + ((y - 1/2) * sigma**2)) * T) / (sigma * np.sqrt(T))
    lam = -r + y*b + 1/2*y*(y - 1)*sigma**2 # lambda calculation
    k = ((2*b) / sigma**2) + (2*y - 1)

    phi = np.exp(lam * T) * (S_0**y) * (norm.cdf(d) - (I / S_0)**k * norm.cdf(d_2))

    return phi


In [ ]:
### Compute Bjerkson_Stensland Call
